Instalación

In [ ]:
!pip install pennylane --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 36.4 MB/s eta 0:00:00


Imports

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import numpy as onp   # para operaciones clásicas donde prefiero numpy "puro"
from math import pi
import matplotlib.pyplot as plt

Configuración

In [ ]:
n_qubits = 4   # Imagen 2x2 → 4 píxeles → 4 qubits
dev = qml.device("default.qubit", wires=n_qubits)

Helpers: traza parcial y Bloch

In [ ]:
def partial_trace_statevector(statevec, keep, dims):
    """
    statevec: vector de amplitudes (complejo) de dimensión prod(dims)
    keep: lista de índices de qubits a conservar (ej: [0])
    dims: lista de dimensiones por subsistema (ej: [2,2,2,2])
    Retorna la matriz densidad reducida de los subsistemas 'keep'.
    """
    # Convertimos a matriz densidad pura completa: |psi><psi|
    psi = statevec.reshape([d for d in dims])
    rho_full = onp.tensordot(psi, onp.conjugate(psi), axes=0)  # tensor de orden 2N
    # rho_full indices: (i0,i1,...,iN, j0,j1,...,jN)
    N = len(dims)
    # Vamos a reordenar e implementar la traza parcial:
    # Convertimos a matriz con índices ((i0,...,iN),(j0,...,jN))
    rho_full = rho_full.reshape((2**N, 2**N))
    # Ahora calculamos la traza parcial con una rutina simple usando tensordot sobre índices
    # Pero para tamaños pequeños (4 qubits) podemos usar la función de qml.math.reduce_statevector? No,
    # implementamos una función genérica: construir la matriz reducida mediante sumas sobre índices a trazar.
    keep = list(keep)
    trace_out = [i for i in range(N) if i not in keep]
    # Construcción por índices (ineficiente pero clara)
    # Usamos representación en base computational: índices 0..2^N-1
    dim = 2**N
    keep_dim = 2**len(keep)
    rho_red = onp.zeros((keep_dim, keep_dim), dtype=complex)
    for a in range(dim):
        for b in range(dim):
            # convertir a,b a bits
            abase = [(a >> k) & 1 for k in range(N)][::-1]  # msb first
            bbase = [(b >> k) & 1 for k in range(N)][::-1]
            # verificamos que los qubits que queremos conservar tengan los mismos índices?
            # Para traza parcial, sumamos sobre los índices de los trazados cuando éstos coinciden
            matches_trace_out = True
            for k in trace_out:
                if abase[k] != bbase[k]:
                    matches_trace_out = False
                    break
            if not matches_trace_out:
                continue
            # si coincide en los índices trazados, contribuimos a la posición correspondiente en rho_red
            # compute index in reduced basis for kept qubits
            idx_a = 0
            idx_b = 0
            for pos, q in enumerate(keep):
                idx_a = (idx_a << 1) | abase[q]
                idx_b = (idx_b << 1) | bbase[q]
            rho_red[idx_a, idx_b] += (statevec[a] * onp.conjugate(statevec[b]))
    return rho_red

In [ ]:
def bloch_from_rho(rho):
    """
    rho: 2x2 density matrix
    devuelve Bloch vector (rx, ry, rz)
    """
    rho00 = rho[0,0]
    rho11 = rho[1,1]
    rho01 = rho[0,1]
    rx = 2 * onp.real(rho01)
    ry = -2 * onp.imag(rho01)
    rz = onp.real(rho00 - rho11)
    return onp.array([rx, ry, rz])

Capas del circuito (cada capa en su propia función)

In [ ]:
def encode_image(image):
    """
    image: array 2x2 con valores de 0..255 (o 0..1).
    Codifica cada píxel en RY(pi * norm_pixel).
    """
    flat = onp.array(image).flatten().astype(float)
    if flat.max() > 1.0:
        flat = flat / 255.0
    for i, pix in enumerate(flat):
        qml.RY(pi * pix, wires=i)


In [ ]:
def ansatz_rotations(params):
    # params: array de tamaño (n_qubits,)
    for i in range(n_qubits):
        qml.RX(params[i], wires=i)
        qml.RZ(params[i], wires=i)

In [ ]:
def entanglement_chain():
    # CNOT en cadena 0->1, 1->2, 2->3
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i+1])

QNodes que retornan el statevector hasta cada punto

In [ ]:
@qml.qnode(dev)
def state_after_encode(image):
    # inicio |0000>
    encode_image(image)
    return qml.state()

In [ ]:
@qml.qnode(dev)
def state_after_rotations(image, params):
    encode_image(image)
    ansatz_rotations(params)
    return qml.state()

In [ ]:
@qml.qnode(dev)
def state_after_entanglement(image, params):
    encode_image(image)
    ansatz_rotations(params)
    entanglement_chain()
    return qml.state()

In [ ]:
@qml.qnode(dev)
def measurements(image, params):
    encode_image(image)
    ansatz_rotations(params)
    entanglement_chain()
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

Función para imprimir resumen de estado

In [ ]:
def print_state_info(title, statevec):
    print("------", title, "------")
    # Imprimir statevector (amplitudes)
    onp.set_printoptions(precision=4, suppress=True)
    print("Statevector (ampitudes) length:", len(statevec))
    for idx, amp in enumerate(statevec):
        if abs(amp) > 1e-6:
            bin_idx = format(idx, '0{}b'.format(n_qubits))
            print(f"|{bin_idx}⟩: {amp}")
    # Bloch de cada qubit
    print("\nBloch vectors por qubit (rx, ry, rz):")
    for q in range(n_qubits):
        rho_q = partial_trace_statevector(statevec, keep=[q], dims=[2]*n_qubits)
        bloch = bloch_from_rho(rho_q)
        print(f"q{q}: {bloch.round(4)}")
    print("\n")


Ejemplo: imagen y parámetros

In [ ]:
image_example = onp.array([[0, 128], [200, 255]])   # pixeles 0..255
params = onp.random.uniform(0, pi, size=(n_qubits,))

Imprimir circuito (dibujado) para referencia

In [ ]:
@qml.qnode(dev)
def circuit_for_draw(image, params):
    encode_image(image)
    ansatz_rotations(params)
    entanglement_chain()
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

In [ ]:
print("Dibujo del circuito (capas en orden):\n")
print(qml.draw(circuit_for_draw)(image_example, params))
print("\n\n")

Dibujo del circuito (capas en orden):

0: ──RY(0.00)──RX(1.93)──RZ(1.93)─╭●───────┤  <Z>
1: ──RY(1.58)──RX(2.63)──RZ(2.63)─╰X─╭●────┤  <Z>
2: ──RY(2.46)──RX(2.33)──RZ(2.33)────╰X─╭●─┤  <Z>
3: ──RY(3.14)──RX(1.79)──RZ(1.79)───────╰X─┤  <Z>





Ejecutar y mostrar estados

In [ ]:
sv_initial = onp.zeros(2**n_qubits, dtype=complex)
sv_initial[0] = 1.0  # |0000>
print_state_info("Estado inicial |0000>", sv_initial)

------ Estado inicial |0000> ------
Statevector (ampitudes) length: 16
|0000⟩: (1+0j)

Bloch vectors por qubit (rx, ry, rz):
q0: [ 0. -0.  1.]
q1: [ 0. -0.  1.]
q2: [ 0. -0.  1.]
q3: [ 0. -0.  1.]




In [ ]:
sv_enc = state_after_encode(image_example)
print_state_info("Después de la capa de CODIFICACIÓN (RY por píxel)", sv_enc)

------ Después de la capa de CODIFICACIÓN (RY por píxel) ------
Statevector (ampitudes) length: 16
|0001⟩: (0.23428538879008193+0j)
|0011⟩: (0.6648536555366232+0j)
|0101⟩: (0.2357330467640142+0j)
|0111⟩: (0.6689618105560469+0j)

Bloch vectors por qubit (rx, ry, rz):
q0: [ 0. -0.  1.]
q1: [ 1.     -0.     -0.0062]
q2: [ 0.6269 -0.     -0.7791]
q3: [ 0. -0. -1.]




In [ ]:
sv_rot = state_after_rotations(image_example, params)
print_state_info("Después de la capa de ROTACIONES variacionales (ansatz_rotations)", sv_rot)

------ Después de la capa de ROTACIONES variacionales (ansatz_rotations) ------
Statevector (ampitudes) length: 16
|0000⟩: (-0.19773305735136765-0.1926947145811963j)
|0001⟩: (0.12158652695377618+0.18536371631903892j)
|0010⟩: (0.11603988175314785+0.09702665213534217j)
|0011⟩: (-0.07413849466525832-0.09619343150737311j)
|0100⟩: (0.2649159638971547+0.07234231085631718j)
|0101⟩: (-0.19513201142676712-0.10266745952062717j)
|0110⟩: (-0.14772687023461553-0.028484438825135003j)
|0111⟩: (0.11086947103648773+0.04795639487761357j)
|1000⟩: (-0.16946016376938333-0.3607522518384031j)
|1001⟩: (0.07026923853213352+0.3122081309025834j)
|1010⟩: (0.10759488284530555+0.1900078634768902j)
|1011⟩: (-0.05138869808447608-0.16762120350061505j)
|1100⟩: (0.32132290616501064+0.23218649067645167j)
|1101⟩: (-0.21162432689203703-0.23776230637891999j)
|1110⟩: (-0.185197281780777-0.11345215468215748j)
|1111⟩: (0.12550519787775097+0.12106686612469872j)

Bloch vectors por qubit (rx, ry, rz):
q0: [ 0.8765  0.3291 -0.3515

In [ ]:
sv_ent = state_after_entanglement(image_example, params)
print_state_info("Después de la capa de ENTRELZAMIENTO (CNOT chain)", sv_ent)

------ Después de la capa de ENTRELZAMIENTO (CNOT chain) ------
Statevector (ampitudes) length: 16
|0000⟩: (-0.19773305735136765-0.1926947145811963j)
|0001⟩: (0.12158652695377618+0.18536371631903892j)
|0010⟩: (-0.07413849466525832-0.09619343150737311j)
|0011⟩: (0.11603988175314785+0.09702665213534217j)
|0100⟩: (-0.14772687023461553-0.028484438825135003j)
|0101⟩: (0.11086947103648773+0.04795639487761357j)
|0110⟩: (-0.19513201142676712-0.10266745952062717j)
|0111⟩: (0.2649159638971547+0.07234231085631718j)
|1000⟩: (0.32132290616501064+0.23218649067645167j)
|1001⟩: (-0.21162432689203703-0.23776230637891999j)
|1010⟩: (0.12550519787775097+0.12106686612469872j)
|1011⟩: (-0.185197281780777-0.11345215468215748j)
|1100⟩: (0.10759488284530555+0.1900078634768902j)
|1101⟩: (-0.05138869808447608-0.16762120350061505j)
|1110⟩: (0.07026923853213352+0.3122081309025834j)
|1111⟩: (-0.16946016376938333-0.3607522518384031j)

Bloch vectors por qubit (rx, ry, rz):
q0: [-0.7667 -0.2878 -0.3515]
q1: [ 0.735   

In [ ]:
meas = measurements(image_example, params)
print("Mediciones finales (⟨Z⟩ por qubit):", onp.round(onp.array(meas), 4))

Mediciones finales (⟨Z⟩ por qubit): [-0.3515 -0.0019 -0.001  -0.0002]
